# Formula 1 Analytics: Comprehensive Performance Analysis

This notebook provides a complete analysis of Formula 1 historical data, uncovering insights about drivers, constructors, pit stop strategies, and race outcomes.

## Objectives
1. **Data Preparation**: Load, clean, and merge F1 datasets
2. **Driver & Constructor Analysis**: Identify top performers by wins, podiums, and consistency
3. **Custom Metrics**: Calculate Consistency Index, Pit Stop Efficiency, and Overtake Index
4. **Trend Analysis**: Examine performance evolution over seasons
5. **Strategic Insights**: Analyze qualifying vs. race performance, pit stop impact
6. **Visualization**: Create compelling charts and dashboards


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.append(str(Path("..").resolve()))

from src.data_cleaner import F1DataCleaner

# Set style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (14, 8)
plt.rcParams["font.size"] = 10

# Color palette
F1_COLORS = ['#E10600', '#1E41FF', '#00D2BE', '#FF8700', '#FFFFFF', '#0090FF', '#FFB800', '#006F62']
sns.set_palette(F1_COLORS)

print("Libraries imported successfully!")


## 1. Data Preparation

Load and clean all datasets, handle missing values, and merge key tables.


In [ ]:
# Load and clean data
data_dir = Path("../data")
cleaner = F1DataCleaner(data_dir=data_dir)

# Load all data
cleaner.load_data()

# Clean all datasets
merged_df, aggregated = cleaner.clean_all()

# Display basic info
print(f"\nMerged dataset shape: {merged_df.shape}")
print(f"\nColumns: {list(merged_df.columns)}")
print(f"\nDate range: {merged_df['year'].min()} - {merged_df['year'].max()}")
print(f"\nTotal races: {merged_df['raceId'].nunique()}")
print(f"\nTotal drivers: {merged_df['driverId'].nunique()}")
print(f"\nTotal constructors: {merged_df['constructorId'].nunique()}")

merged_df.head()


In [ ]:
# Data quality check
print("Missing values per column:")
print(merged_df.isnull().sum().sort_values(ascending=False))
print(f"\nData completeness: {(1 - merged_df.isnull().sum().sum() / (merged_df.shape[0] * merged_df.shape[1])) * 100:.2f}%")


## 2. Top Drivers and Constructors Analysis


In [ ]:
# Top drivers by wins
driver_stats = aggregated['driver_stats'].copy()
top_drivers_wins = driver_stats.nlargest(15, 'wins')[['full_name', 'wins', 'podiums', 'total_points', 'win_rate', 'podium_rate']]

print("🏆 Top 15 Drivers by Wins:")
print(top_drivers_wins.to_string(index=False))

# Top drivers by podiums
top_drivers_podiums = driver_stats.nlargest(15, 'podiums')[['full_name', 'wins', 'podiums', 'total_points', 'podium_rate']]

print("\n🏁 Top 15 Drivers by Podiums:")
print(top_drivers_podiums.to_string(index=False))


In [ ]:
# Top constructors by wins
constructor_stats = aggregated['constructor_stats'].copy()
top_constructors = constructor_stats.nlargest(15, 'wins')[['name', 'wins', 'podiums', 'total_points', 'win_rate', 'podium_rate']]

print("🏎️ Top 15 Constructors by Wins:")
print(top_constructors.to_string(index=False))


In [ ]:
# Visualization: Top drivers by wins
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Top 10 drivers by wins
top_10_wins = driver_stats.nlargest(10, 'wins')
axes[0].barh(top_10_wins['full_name'], top_10_wins['wins'], color='#E10600')
axes[0].set_xlabel('Number of Wins', fontsize=12, fontweight='bold')
axes[0].set_title('Top 10 Drivers by Race Wins', fontsize=14, fontweight='bold')
axes[0].invert_yaxis()
axes[0].grid(axis='x', alpha=0.3)

# Top 10 drivers by podiums
top_10_podiums = driver_stats.nlargest(10, 'podiums')
axes[1].barh(top_10_podiums['full_name'], top_10_podiums['podiums'], color='#1E41FF')
axes[1].set_xlabel('Number of Podiums', fontsize=12, fontweight='bold')
axes[1].set_title('Top 10 Drivers by Podium Finishes', fontsize=14, fontweight='bold')
axes[1].invert_yaxis()
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Visualization: Top constructors
fig, ax = plt.subplots(figsize=(14, 8))
top_10_constructors = constructor_stats.nlargest(10, 'wins')
ax.barh(top_10_constructors['name'], top_10_constructors['wins'], color='#00D2BE')
ax.set_xlabel('Number of Wins', fontsize=12, fontweight='bold')
ax.set_title('Top 10 Constructors by Race Wins', fontsize=14, fontweight='bold')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()


## 3. Custom Metrics Calculation

### 3.1 Consistency Index
Consistency Index = std(lap_time) per driver per race (lower is better - more consistent)


In [ ]:
# Calculate Consistency Index using lap times
if cleaner.laptimes is not None and not cleaner.laptimes.empty:
    # Calculate standard deviation of lap times per driver per race
    consistency = cleaner.laptimes.groupby(['raceId', 'year', 'driverId'])['time_seconds'].std().reset_index()
    consistency.columns = ['raceId', 'year', 'driverId', 'consistency_index']
    
    # Merge with driver names
    consistency = consistency.merge(
        cleaner.drivers[['driverId', 'full_name']],
        on='driverId',
        how='left'
    )
    
    # Calculate average consistency per driver
    driver_consistency = consistency.groupby('full_name')['consistency_index'].mean().reset_index()
    driver_consistency = driver_consistency.sort_values('consistency_index')
    
    print("📊 Driver Consistency Index (Lower = More Consistent):")
    print(driver_consistency.head(15).to_string(index=False))
else:
    # Alternative: Use position variance as consistency measure
    print("Lap time data not available. Using position variance as consistency measure...")
    position_consistency = merged_df.groupby(['driverId', 'year'])['position'].std().reset_index()
    position_consistency.columns = ['driverId', 'year', 'position_std']
    
    driver_consistency = position_consistency.groupby('driverId')['position_std'].mean().reset_index()
    driver_consistency = driver_consistency.merge(
        cleaner.drivers[['driverId', 'full_name']],
        on='driverId',
        how='left'
    )
    driver_consistency = driver_consistency.sort_values('position_std')
    driver_consistency.columns = ['driverId', 'consistency_index', 'full_name']
    
    print("📊 Driver Consistency Index (Lower Position Variance = More Consistent):")
    print(driver_consistency.head(15)[['full_name', 'consistency_index']].to_string(index=False))


### 3.2 Pit Stop Efficiency
Pit Stop Efficiency = total_pit_time / final_position_change (lower is better)


In [ ]:
# Calculate Pit Stop Efficiency
if cleaner.pitstops is not None and not cleaner.pitstops.empty:
    # Aggregate pit stop times per driver per race
    pit_agg = cleaner.pitstops.groupby(['raceId', 'year', 'driverId']).agg({
        'duration_seconds': 'sum',
        'stop': 'count'
    }).reset_index()
    pit_agg.columns = ['raceId', 'year', 'driverId', 'total_pit_time', 'num_pitstops']
    
    # Merge with results to get position change
    pit_efficiency = pit_agg.merge(
        merged_df[['raceId', 'year', 'driverId', 'position_change', 'grid', 'position']],
        on=['raceId', 'year', 'driverId'],
        how='inner'
    )
    
    # Calculate efficiency: total_pit_time / position_change
    # Position change > 0 means gained positions (good)
    pit_efficiency['position_change_abs'] = abs(pit_efficiency['position_change'])
    pit_efficiency['pit_efficiency'] = np.where(
        pit_efficiency['position_change_abs'] > 0,
        pit_efficiency['total_pit_time'] / pit_efficiency['position_change_abs'],
        pit_efficiency['total_pit_time']  # If no position change, use total time
    )
    
    # Average efficiency per driver
    driver_pit_efficiency = pit_efficiency.groupby('driverId').agg({
        'pit_efficiency': 'mean',
        'total_pit_time': 'mean',
        'position_change': 'mean'
    }).reset_index()
    
    driver_pit_efficiency = driver_pit_efficiency.merge(
        cleaner.drivers[['driverId', 'full_name']],
        on='driverId',
        how='left'
    )
    driver_pit_efficiency = driver_pit_efficiency.sort_values('pit_efficiency')
    
    print("⏱️ Pit Stop Efficiency (Lower = More Efficient):")
    print(driver_pit_efficiency.head(15)[['full_name', 'pit_efficiency', 'total_pit_time', 'position_change']].to_string(index=False))
else:
    print("Pit stop data not available for this analysis")


### 3.3 Overtake Index
Overtake Index = positions_gained_after_first_lap


In [ ]:
# Calculate Overtake Index (positions gained from grid to finish)
overtake_data = merged_df[['driverId', 'full_name', 'grid', 'position', 'position_change', 'year']].copy()
overtake_data = overtake_data.dropna(subset=['grid', 'position'])

# Overtake index = position_change (positive means gained positions)
overtake_data['overtake_index'] = overtake_data['position_change']

# Aggregate per driver
driver_overtake = overtake_data.groupby('driverId').agg({
    'overtake_index': 'mean',
    'position_change': ['mean', 'sum', 'count']
}).reset_index()
driver_overtake.columns = ['driverId', 'avg_overtake_index', 'avg_position_change', 'total_positions_gained', 'races']
driver_overtake = driver_overtake.merge(
    cleaner.drivers[['driverId', 'full_name']],
    on='driverId',
    how='left'
)
driver_overtake = driver_overtake.sort_values('avg_overtake_index', ascending=False)

print("🚗 Overtake Index (Higher = More Positions Gained on Average):")
print(driver_overtake.head(15)[['full_name', 'avg_overtake_index', 'total_positions_gained', 'races']].to_string(index=False))


## 4. Performance Trends Over Time


In [ ]:
# Driver performance trends - Top drivers over time
top_drivers_list = driver_stats.nlargest(10, 'wins')['driverId'].tolist()

driver_trends = merged_df[merged_df['driverId'].isin(top_drivers_list)].copy()
driver_trends = driver_trends.groupby(['year', 'driverId', 'full_name']).agg({
    'position': 'mean',
    'points': 'sum',
    'wins': lambda x: (x == 1).sum() if 'wins' in driver_trends.columns else 0
}).reset_index()

# Calculate wins per year
driver_trends['wins'] = merged_df[merged_df['driverId'].isin(top_drivers_list)].groupby(['year', 'driverId'])['position'].apply(lambda x: (x == 1).sum()).reset_index()['position']

fig, axes = plt.subplots(2, 1, figsize=(16, 12))

# Average position over time
for driver_id in top_drivers_list[:5]:  # Top 5 for clarity
    driver_name = cleaner.drivers[cleaner.drivers['driverId'] == driver_id]['full_name'].values[0]
    driver_data = driver_trends[driver_trends['driverId'] == driver_id]
    axes[0].plot(driver_data['year'], driver_data['position'], marker='o', label=driver_name, linewidth=2)

axes[0].set_xlabel('Year', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Average Position', fontsize=12, fontweight='bold')
axes[0].set_title('Average Race Position Over Time (Top 5 Drivers)', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)
axes[0].invert_yaxis()

# Points per year
for driver_id in top_drivers_list[:5]:
    driver_name = cleaner.drivers[cleaner.drivers['driverId'] == driver_id]['full_name'].values[0]
    driver_data = driver_trends[driver_trends['driverId'] == driver_id]
    axes[1].plot(driver_data['year'], driver_data['points'], marker='s', label=driver_name, linewidth=2)

axes[1].set_xlabel('Year', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Total Points', fontsize=12, fontweight='bold')
axes[1].set_title('Total Points Per Season (Top 5 Drivers)', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Constructor performance trends
top_constructors_list = constructor_stats.nlargest(5, 'wins')['constructorId'].tolist()

constructor_trends = merged_df[merged_df['constructorId'].isin(top_constructors_list)].copy()
constructor_trends = constructor_trends.groupby(['year', 'constructorId', 'constructor_name']).agg({
    'points': 'sum',
    'position': lambda x: (x == 1).sum()  # Wins
}).reset_index()
constructor_trends.columns = ['year', 'constructorId', 'constructor_name', 'points', 'wins']

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Points over time
for constructor_id in top_constructors_list:
    constructor_name = cleaner.constructors[cleaner.constructors['constructorId'] == constructor_id]['name'].values[0]
    constructor_data = constructor_trends[constructor_trends['constructorId'] == constructor_id]
    axes[0].plot(constructor_data['year'], constructor_data['points'], marker='o', label=constructor_name, linewidth=2)

axes[0].set_xlabel('Year', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Total Points', fontsize=12, fontweight='bold')
axes[0].set_title('Constructor Points Over Time', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Wins over time
for constructor_id in top_constructors_list:
    constructor_name = cleaner.constructors[cleaner.constructors['constructorId'] == constructor_id]['name'].values[0]
    constructor_data = constructor_trends[constructor_trends['constructorId'] == constructor_id]
    axes[1].bar(constructor_data['year'], constructor_data['wins'], label=constructor_name, alpha=0.7)

axes[1].set_xlabel('Year', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Number of Wins', fontsize=12, fontweight='bold')
axes[1].set_title('Constructor Wins Over Time', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


## 5. Qualifying vs. Race Performance


In [ ]:
# Analyze correlation between qualifying position and final position
qual_race_data = merged_df[['qualifying_position', 'position', 'grid', 'full_name', 'year']].copy()
qual_race_data = qual_race_data.dropna(subset=['qualifying_position', 'position'])

# Calculate correlation
correlation = qual_race_data['qualifying_position'].corr(qual_race_data['position'])
print(f"📊 Correlation between Qualifying Position and Final Position: {correlation:.3f}")
print("(Higher correlation = qualifying position is a strong predictor of race finish)")

# Scatter plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.scatter(qual_race_data['qualifying_position'], qual_race_data['position'], 
           alpha=0.3, s=20, color='#E10600')
ax.plot([1, 20], [1, 20], 'k--', linewidth=2, label='Perfect Correlation')
ax.set_xlabel('Qualifying Position', fontsize=12, fontweight='bold')
ax.set_ylabel('Final Race Position', fontsize=12, fontweight='bold')
ax.set_title(f'Qualifying Position vs. Final Race Position\n(Correlation: {correlation:.3f})', 
             fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)
ax.invert_xaxis()
ax.invert_yaxis()
plt.tight_layout()
plt.show()


In [ ]:
# Calculate average position change by qualifying position
qual_race_data['position_change'] = qual_race_data['qualifying_position'] - qual_race_data['position']
avg_change_by_qual = qual_race_data.groupby('qualifying_position')['position_change'].mean().reset_index()

fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(avg_change_by_qual['qualifying_position'], avg_change_by_qual['position_change'], 
       color='#1E41FF', alpha=0.7)
ax.axhline(y=0, color='black', linestyle='--', linewidth=1)
ax.set_xlabel('Qualifying Position', fontsize=12, fontweight='bold')
ax.set_ylabel('Average Position Change', fontsize=12, fontweight='bold')
ax.set_title('Average Position Change by Qualifying Position\n(Positive = Gained positions, Negative = Lost positions)', 
             fontsize=14, fontweight='bold')
ax.invert_xaxis()
ax.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.show()


## 6. Pit Stop Analysis


In [ ]:
# Pit stop duration distribution
if cleaner.pitstops is not None and not cleaner.pitstops.empty:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Distribution of pit stop durations
    pit_durations = cleaner.pitstops['duration_seconds'].dropna()
    axes[0].hist(pit_durations, bins=50, color='#00D2BE', alpha=0.7, edgecolor='black')
    axes[0].axvline(pit_durations.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {pit_durations.mean():.2f}s')
    axes[0].set_xlabel('Pit Stop Duration (seconds)', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Frequency', fontsize=12, fontweight='bold')
    axes[0].set_title('Distribution of Pit Stop Durations', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3, axis='y')
    
    # Average pit stops per race over time
    pit_by_year = cleaner.pitstops.groupby('year').agg({
        'duration_seconds': 'mean',
        'stop': 'count'
    }).reset_index()
    pit_by_year.columns = ['year', 'avg_duration', 'total_stops']
    
    axes[1].plot(pit_by_year['year'], pit_by_year['avg_duration'], marker='o', linewidth=2, color='#E10600')
    axes[1].set_xlabel('Year', fontsize=12, fontweight='bold')
    axes[1].set_ylabel('Average Duration (seconds)', fontsize=12, fontweight='bold')
    axes[1].set_title('Average Pit Stop Duration Over Time', fontsize=14, fontweight='bold')
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"📊 Pit Stop Statistics:")
    print(f"Average duration: {pit_durations.mean():.2f} seconds")
    print(f"Median duration: {pit_durations.median():.2f} seconds")
    print(f"Fastest pit stop: {pit_durations.min():.2f} seconds")
    print(f"Slowest pit stop: {pit_durations.max():.2f} seconds")
else:
    print("Pit stop data not available")


## 7. DNF (Did Not Finish) Analysis


In [ ]:
# DNF rate analysis
dnf_analysis = merged_df.groupby(['year']).agg({
    'is_dnf': ['sum', 'mean'],
    'driverId': 'count'
}).reset_index()
dnf_analysis.columns = ['year', 'total_dnfs', 'dnf_rate', 'total_drivers']

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# DNF rate over time
axes[0].plot(dnf_analysis['year'], dnf_analysis['dnf_rate'] * 100, marker='o', linewidth=2, color='#E10600')
axes[0].set_xlabel('Year', fontsize=12, fontweight='bold')
axes[0].set_ylabel('DNF Rate (%)', fontsize=12, fontweight='bold')
axes[0].set_title('DNF Rate Over Time', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)

# DNF by driver
driver_dnf = merged_df.groupby('full_name').agg({
    'is_dnf': ['sum', 'mean', 'count']
}).reset_index()
driver_dnf.columns = ['driver', 'total_dnfs', 'dnf_rate', 'races']
driver_dnf = driver_dnf[driver_dnf['races'] >= 20].sort_values('dnf_rate', ascending=False).head(10)

axes[1].barh(driver_dnf['driver'], driver_dnf['dnf_rate'] * 100, color='#FF8700')
axes[1].set_xlabel('DNF Rate (%)', fontsize=12, fontweight='bold')
axes[1].set_title('Top 10 Drivers by DNF Rate (Min 20 races)', fontsize=14, fontweight='bold')
axes[1].invert_yaxis()
axes[1].grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.show()


## 8. Circuit Analysis


In [ ]:
# Circuit-wise analysis
if 'circuit_name' in merged_df.columns:
    circuit_stats = merged_df.groupby('circuit_name').agg({
        'raceId': 'nunique',
        'position': 'mean',
        'points': 'sum'
    }).reset_index()
    circuit_stats.columns = ['circuit', 'races', 'avg_position', 'total_points']
    circuit_stats = circuit_stats.sort_values('races', ascending=False).head(15)
    
    fig, ax = plt.subplots(figsize=(14, 8))
    ax.barh(circuit_stats['circuit'], circuit_stats['races'], color='#0090FF')
    ax.set_xlabel('Number of Races Hosted', fontsize=12, fontweight='bold')
    ax.set_title('Top 15 Circuits by Number of Races Hosted', fontsize=14, fontweight='bold')
    ax.invert_yaxis()
    ax.grid(alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()


## 9. Key Performance Indicators (KPIs)

Calculate and display key metrics for drivers and constructors.


In [ ]:
# Calculate KPIs
print("=" * 80)
print("KEY PERFORMANCE INDICATORS")
print("=" * 80)

# Overall statistics
total_races = merged_df['raceId'].nunique()
total_drivers = merged_df['driverId'].nunique()
total_constructors = merged_df['constructorId'].nunique()

print(f"\n📊 Overall Statistics:")
print(f"   Total Races: {total_races:,}")
print(f"   Total Drivers: {total_drivers:,}")
print(f"   Total Constructors: {total_constructors:,}")

# Top driver KPIs
top_driver = driver_stats.nlargest(1, 'wins').iloc[0]
print(f"\n🏆 Top Driver (by wins): {top_driver['full_name']}")
print(f"   Wins: {int(top_driver['wins'])}")
print(f"   Podiums: {int(top_driver['podiums'])}")
print(f"   Win Rate: {top_driver['win_rate']*100:.2f}%")
print(f"   Podium Rate: {top_driver['podium_rate']*100:.2f}%")
print(f"   Total Points: {int(top_driver['total_points'])}")

# Top constructor KPIs
top_constructor = constructor_stats.nlargest(1, 'wins').iloc[0]
print(f"\n🏎️ Top Constructor (by wins): {top_constructor['name']}")
print(f"   Wins: {int(top_constructor['wins'])}")
print(f"   Podiums: {int(top_constructor['podiums'])}")
print(f"   Win Rate: {top_constructor['win_rate']*100:.2f}%")
print(f"   Podium Rate: {top_constructor['podium_rate']*100:.2f}%")

# Pit stop KPIs
if cleaner.pitstops is not None and not cleaner.pitstops.empty:
    avg_pit_duration = cleaner.pitstops['duration_seconds'].mean()
    print(f"\n⏱️ Pit Stop Statistics:")
    print(f"   Average Duration: {avg_pit_duration:.2f} seconds")
    print(f"   Total Pit Stops: {len(cleaner.pitstops):,}")

# DNF statistics
overall_dnf_rate = merged_df['is_dnf'].mean() * 100
print(f"\n⚠️ DNF Statistics:")
print(f"   Overall DNF Rate: {overall_dnf_rate:.2f}%")
print(f"   Total DNFs: {merged_df['is_dnf'].sum():,}")

print("\n" + "=" * 80)
